# Part 4 - Forward Synchronization

to come.

### Files 
1. Notebook "Part 4 - Forward Syncronization"
1. Notebook "OSC Server"
2. Metronome tick "./tick.wav"
3. Pure Data patch from "../pd/part4_forward_synchronization.pd"
4. Metronome tick "../pd/tick.wav"
5. Notebook "Extra - UTC Timestamp Printing"

<p align="left">
 <img src="../../fig/example4.jpg" width=500>
</p>

In [1]:
from pythonosc import udp_client
from pythonosc import osc_bundle_builder, osc_message_builder
import time
import threading

In [2]:
# Config
BPM = 100  # Beats per minute
BEAT_INTERVAL = 60 / BPM

# seconds into the future to schedule ticks
FORWARD_OFFSET = 4  

## Define Remote OSC Clients (Pure Data and Python)

In [3]:
# Remote client (Pure Data)
clientIp = '127.0.0.1'
clientPort = 8001
client1 = udp_client.UDPClient(clientIp, clientPort)

# Local client (Python server)
client2Ip = '127.0.0.1'
client2Port = 8000
client2 = udp_client.UDPClient(client2Ip, client2Port)

## Message sending logic

In [ ]:
stop_threading = False

def sendMsg(clients):
    print(f'Sending messages to client.')
    
    start_time = time.time()
    beat_number = 0

    while not stop_threading:
        # NEW custom timestamp = start time + current while Loop round /tick_numb) + forward sync offset
        custom_timestamp = start_time + (beat_number * BEAT_INTERVAL) + FORWARD_OFFSET

        # Create bundle with future timestamp
        bundle = osc_bundle_builder.OscBundleBuilder(custom_timestamp)

        # Create and add message
        msg = osc_message_builder.OscMessageBuilder(address="/tick")
        
        msg.add_arg("tick")
        
        bundle.add_content(msg.build())

        # Send the bundle
        clients[0].send(bundle.build())
        clients[1].send(bundle.build())

        # increment the beat number for every loop
        beat_number += 1

        # Wait for the next beat
        time.sleep(BEAT_INTERVAL)

## Send Messages to Remote Clients

In [ ]:
if __name__ == "__main__":
    # Start sending messages to two identical clients (local and remote), but in individual threads
    t_clients = threading.Thread(target=sendMsg, args=([client1, client2],), daemon=True)

    # start sending
    t_clients.start()

    print("Press Ctrl+C to exit.")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Exiting...")

Sending messages to client.
Sending messages to client.
Press Ctrl+C to exit.
Exiting...


In [10]:
stop_threading = True